In [1]:
import os
CURRENTPATH = os.getcwd()
NANOSHAPERCMD = "E:\\Tesis\\Nanoshaper\\Bugfix\\bin\\NanoShaper64.exe"
ITER = 20
alpha = 0.5
beta = -60
gamma = -0.5
MAXITER_GM = 1000
TOL = 1e-8
f_results = open('Results_Amino_acids.txt','a')

In [2]:
# Importing the neccesary modules:
import bempp.api
import numpy as np
from scipy.sparse.linalg import gmres
import shutil

In [3]:
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import matplotlib as mpl
from matplotlib import pyplot as plt
from numba import njit
@njit
def impl1(F):
    for i in range(F.shape[0]):
        nrm = np.linalg.norm(F[i,:3])
        F[i] = nrm
    return F[:,0]


def plot_mesh_to_file(directory,grid): #USAR PATH COMPLETO DE ARCHIVO
    vertices = grid.vertices
    elements = grid.elements
    fig = ff.create_trisurf(
                x=vertices[0, :],
                y=vertices[1, :],
                z=vertices[2, :],
                simplices=elements.T,
                color_func=elements.shape[1] * ["rgb(255, 222, 173)"],
            )
    fig["layout"]["scene"].update(go.layout.Scene(aspectmode="data"))
    #plotly.offline.iplot(fig)
    fig.write_html(directory+".html")
    fig.write_image(directory+".png",scale =4.0)
    
def plot_solution_to_file(directory,solution): #USAR PATH COMPLETO
    grid = solution.space.grid
    vertices = grid.vertices
    elements = grid.elements
    local_coordinates = np.array([[1.0 / 3], [1.0 / 3]])
    intensity_1 = np.zeros(grid.entity_count(0), dtype="float64")
    for element in grid.entity_iterator(0):
        index = element.index
        local_values = np.real((solution.evaluate(index, local_coordinates)))
        intensity_1[index] = local_values.flatten()

    i_0 = elements[0,:]
    j_0 = elements[1,:]
    k_0 = elements[2,:]

    triangles = np.vstack((i_0,j_0,k_0)).T

    x_0 = vertices[0, :]
    y_0 = vertices[1, :]
    z_0 = vertices[2, :]
    vertices = np.vstack((x_0,y_0,z_0)).T
    tri_points = vertices[triangles]

    #extract the lists of x, y, z coordinates of the triangle vertices and connect them by a line
    Xe = []
    Ye = []
    Ze = []
    for T in tri_points:
        Xe.extend([T[k%3][0] for k in range(4)]+[ None])
        Ye.extend([T[k%3][1] for k in range(4)]+[ None])
        Ze.extend([T[k%3][2] for k in range(4)]+[ None])

    #define the trace for triangle sides
    lines = go.Scatter3d(
                       x=Xe,
                       y=Ye,
                       z=Ze,
                       mode='lines',
                       name='',
                       line=dict(color= 'rgb(0,0,0)', width=1))  

    fig = go.Figure(data=[
        go.Mesh3d(
            # 8 vertices of a cube
            x=x_0,
            y=y_0,
            z=z_0,
            colorscale="Jet",
            # Intensity of each vertex, which will be interpolated and color-coded
            intensity = intensity_1,
            intensitymode='cell',
            # i, j and k give the vertices of triangles
            i = i_0,
            j = j_0,
            k = k_0,
            name='y',
        ),lines
    ])
    fig["layout"]["scene"].update(go.layout.Scene(aspectmode="data"))
    fig.write_html(directory+".html")
    fig.write_image(directory+".png",scale =4.0)

In [4]:
# This function creates bempp grids from .face and .vert files:
def import_msms_mesh(mesh_face_path, mesh_vert_path):
    face = open(mesh_face_path,'r').read()
    vert = open(mesh_vert_path,'r').read()

    faces = np.vstack(np.char.split(face.split('\n')[3:-1]))[:,:3].astype(int) - 1
    verts = np.vstack(np.char.split(vert.split('\n')[3:-1]))[:,:3].astype(float)

    #grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    grid = bempp.api.Grid(verts.transpose(), faces.transpose())
    return grid

def convert_pqr2xyzr(mesh_pqr_path, mesh_xyzr_path):
    pqr_file = open(mesh_pqr_path, 'r')
    pqr_data = pqr_file.read().split('\n')
    xyzr_file = open(mesh_xyzr_path, 'w')
    for line in pqr_data:
        line = line.split()
        if len(line)==0 or line[0]!='ATOM': continue
        xyzr_file.write(line[6]+"\t"+line[7]+"\t"+line[8]+"\t"+line[10]+"\n")
    pqr_file.close()
    xyzr_file.close()
    
def convert_pqr2xyzr_stern(mesh_pqr_path, mesh_stern_xyzr_path,r):
    pqr_file = open(mesh_pqr_path, 'r')
    pqr_data = pqr_file.read().split('\n')
    xyzr_file = open(mesh_stern_xyzr_path, 'w')
    for line in pqr_data:
        line = line.split()
        if len(line)==0 or line[0]!='ATOM': continue
        xyzr_file.write(line[6]+"\t"+line[7]+"\t"+line[8]+"\t"+str(float(line[10])+r)+"\n")
    pqr_file.close()
    xyzr_file.close()

In [5]:
res = ['arg', 'asp', 'cys', 'glu', 'his', 'lys', 'tyr']
oneletter = ['r','d','c','e','h','k','y']

In [ ]:
'''
Grid_scale = 1.2
XYZR_FileName = E:\Tesis\SLIC_5PTI\5pti_stern.xyzr
'''

In [ ]:
os.getcwd()

In [ ]:
r = 2
#2.7
grid_scale = 2.7
for i in [1,2]:
    os.chdir(CURRENTPATH)
    for amino in range(len(res)):
        pqrpath = "E:\\Tesis\\SLIC_Amino_acids\\"+res[amino]+"\\"+"j"+oneletter[amino]+str(i)+".pqr"
        xyzrpath = "E:\\Tesis\\SLIC_Amino_acids\\"+res[amino]+"\\"+"j"+oneletter[amino]+str(i)+".xyzr"
        xyzrsternpath = "E:\\Tesis\\SLIC_Amino_acids\\"+ \
                            res[amino]+"\\"+"j"+oneletter[amino]+str(i)+"_stern.xyzr"
        os.chdir(res[amino])
        aminodir = os.getcwd()
        #Borrar anteriores
        if i == 1:
            test = os.listdir(aminodir)
            for item in test:
                if item.endswith(".face") or item.endswith(".vert"):
                    os.remove(os.path.join(aminodir, item))
                if item.endswith("results"):
                    shutil.rmtree(os.path.join(aminodir, item))
            os.mkdir("results")
        convert_pqr2xyzr(pqrpath, xyzrpath)
        convert_pqr2xyzr_stern(pqrpath,xyzrsternpath,r)
            #DIEL
        with open(CURRENTPATH+"\\config") as f2:
                with open(CURRENTPATH+"\\config"+res[amino], "w") as f3:
                    for line in f2:
                        f3.write(line)
                    f3.write("\nXYZR_FileName ="+xyzrpath+"\n")
                    f3.write("Grid_scale = "+str(grid_scale)+"\n")
        os.system(NANOSHAPERCMD+' '+CURRENTPATH+'\\config'+res[amino])
        outputfilename = "E:\\Tesis\\SLIC_Amino_acids\\"+ \
                            res[amino]+"\\triangulatedSurf"
        os.rename(outputfilename+".face", \
                  "E:\\Tesis\\SLIC_Amino_acids\\"+res[amino]+"\\"+res[amino]+"dielectric"+str(i)+".face")
        os.rename(outputfilename+".vert", \
                  "E:\\Tesis\\SLIC_Amino_acids\\"+res[amino]+"\\"+res[amino]+"dielectric"+str(i)+".vert")
        #STERN
        with open(CURRENTPATH+"\\config") as f2:
                with open(CURRENTPATH+"\\config"+res[amino]+"stern", "w") as f3:
                    for line in f2:
                        f3.write(line)
                    f3.write("\nXYZR_FileName ="+xyzrsternpath+"\n")
                    f3.write("Grid_scale = "+str(grid_scale)+"\n")
        os.system(NANOSHAPERCMD+' '+CURRENTPATH+'\\config'+res[amino]+"stern")
        os.rename(outputfilename+".face", \
                  "E:\\Tesis\\SLIC_Amino_acids\\"+res[amino]+"\\"+res[amino]+"stern"+str(i)+".face")
        os.rename(outputfilename+".vert", \
                  "E:\\Tesis\\SLIC_Amino_acids\\"+res[amino]+"\\"+res[amino]+"stern"+str(i)+".vert")

        os.chdir('..')

In [6]:
for i in [1,2]:
    os.chdir(CURRENTPATH)
    for amino in range(len(res)):
        os.chdir(res[amino])
        for elemento in os.listdir():
            if elemento.endswith('vert'):
                a = open(elemento,'r')
                for linea in a.readlines():
                    if 'IO' in linea:
                        print(elemento)
                a.close()
        os.chdir(CURRENTPATH)
        

In [7]:
def block_diagonal_PB():
    from scipy.sparse import diags, bmat,block_diag
    from scipy.sparse.linalg import aslinearoperator
    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

    # block-diagonal preconditioner
    identity_protein = sparse.identity(dirichlet_protein, dirichlet_protein, dirichlet_protein)
    identity_protein_diag = identity_protein.weak_form().to_sparse().diagonal()
    V_L_I_II_a_a_diag = laplace.single_layer(neumann_protein, dirichlet_protein, dirichlet_protein,
                                       assembler="only_diagonal_part").weak_form().get_diagonal()
    K_L_I_II_a_a_diag = laplace.double_layer(dirichlet_protein, dirichlet_protein, dirichlet_protein,
                                       assembler="only_diagonal_part").weak_form().get_diagonal()
    identity_stern = sparse.identity(dirichlet_stern, dirichlet_stern, dirichlet_stern)
    identity_stern_diag = identity_stern.weak_form().to_sparse().diagonal()
    V_L_II_b_b_diag = laplace.single_layer(neumann_stern, dirichlet_stern, dirichlet_stern,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    K_L_II_b_b_diag = laplace.double_layer(neumann_stern, dirichlet_stern, dirichlet_stern, 
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    V_Y_I_b_b_diag = modified_helmholtz.single_layer(neumann_stern, dirichlet_stern, dirichlet_stern, om,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    K_Y_I_b_b_diag = modified_helmholtz.double_layer(neumann_stern, dirichlet_stern, dirichlet_stern, om,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()

    diag11 = .5 * identity_protein_diag + K_L_I_II_a_a_diag
    diag12 = -V_L_I_II_a_a_diag
    diag21 = .5 * identity_protein_diag - K_L_I_II_a_a_diag
    diag22 = (ep_i / ep_ii) * V_L_I_II_a_a_diag

    d_aux = 1 / (diag22 - diag21 * diag12 / diag11)
    diag11_inv = 1 / diag11 + 1 / diag11 * diag12 * d_aux * diag21 / diag11
    diag12_inv = -1 / diag11 * diag12 * d_aux
    diag21_inv = -d_aux * diag21 / diag11
    diag22_inv = d_aux
    
    diag33 = .5 * identity_stern_diag +  K_L_II_b_b_diag
    diag34 = -V_L_II_b_b_diag
    diag43 = .5 * identity_stern_diag - K_Y_I_b_b_diag
    diag44 = (ep_ii / ep_iii) * V_Y_I_b_b_diag
    
    d_aux = 1 / (diag44 - diag43 * diag34 / diag33)
    diag33_inv = 1 / diag33 + 1 / diag33 * diag34 * d_aux * diag43 / diag33
    diag34_inv = -1 / diag33 * diag34 * d_aux
    diag43_inv = -d_aux * diag43 / diag33
    diag44_inv = d_aux

    A = bmat([[diags(diag11_inv), diags(diag12_inv)],[diags(diag21_inv), diags(diag22_inv)]])
    B = bmat([[diags(diag33_inv), diags(diag34_inv)],[diags(diag43_inv), diags(diag44_inv)]])
              
    block_mat_precond = block_diag((A,B),format = 'csr')#.tocsr()
    return aslinearoperator(block_mat_precond)

def block_diagonal_SLIC():
    from scipy.sparse import diags, bmat,block_diag
    from scipy.sparse.linalg import aslinearoperator
    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

    # block-diagonal preconditioner
    identity_protein = sparse.identity(dirichlet_protein, dirichlet_protein, dirichlet_protein)
    identity_protein_diag = identity_protein.weak_form().to_sparse().diagonal()
    V_L_I_II_a_a_diag = laplace.single_layer(neumann_protein, dirichlet_protein, dirichlet_protein,
                                       assembler="only_diagonal_part").weak_form().get_diagonal()
    K_L_I_II_a_a_diag = laplace.double_layer(dirichlet_protein, dirichlet_protein, dirichlet_protein,
                                       assembler="only_diagonal_part").weak_form().get_diagonal()
    identity_stern = sparse.identity(dirichlet_stern, dirichlet_stern, dirichlet_stern)
    identity_stern_diag = identity_stern.weak_form().to_sparse().diagonal()
    V_L_II_b_b_diag = laplace.single_layer(neumann_stern, dirichlet_stern, dirichlet_stern,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    K_L_II_b_b_diag = laplace.double_layer(neumann_stern, dirichlet_stern, dirichlet_stern, 
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    V_Y_I_b_b_diag = modified_helmholtz.single_layer(neumann_stern, dirichlet_stern, dirichlet_stern, om,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    K_Y_I_b_b_diag = modified_helmholtz.double_layer(neumann_stern, dirichlet_stern, dirichlet_stern, om,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()

    diag11 = .5 * identity_protein_diag + K_L_I_II_a_a_diag
    diag12 = -V_L_I_II_a_a_diag
    diag21 = .5 * identity_protein_diag - K_L_I_II_a_a_diag
    diag22 = V_L_I_II_a_a_diag * f_op.weak_form().to_sparse().diagonal()

    d_aux = 1 / (diag22 - diag21 * diag12 / diag11)
    diag11_inv = 1 / diag11 + 1 / diag11 * diag12 * d_aux * diag21 / diag11
    diag12_inv = -1 / diag11 * diag12 * d_aux
    diag21_inv = -d_aux * diag21 / diag11
    diag22_inv = d_aux
    
    diag33 = .5 * identity_stern_diag +  K_L_II_b_b_diag
    diag34 = -V_L_II_b_b_diag 
    diag43 = .5 * identity_stern_diag - K_Y_I_b_b_diag
    diag44 = (d1 / d2) * V_Y_I_b_b_diag
    
    d_aux = 1 / (diag44 - diag43 * diag34 / diag33)
    diag33_inv = 1 / diag33 + 1 / diag33 * diag34 * d_aux * diag43 / diag33
    diag34_inv = -1 / diag33 * diag34 * d_aux
    diag43_inv = -d_aux * diag43 / diag33
    diag44_inv = d_aux

    A = bmat([[diags(diag11_inv), diags(diag12_inv)],[diags(diag21_inv), diags(diag22_inv)]])
    B = bmat([[diags(diag33_inv), diags(diag34_inv)],[diags(diag43_inv), diags(diag44_inv)]])
              
    block_mat_precond = block_diag((A,B),format = 'csr')#.tocsr()
    return aslinearoperator(block_mat_precond)

def block_diagonal_SLIC_PROP():
    from scipy.sparse import diags, bmat,block_diag
    from scipy.sparse.linalg import aslinearoperator
    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

    # block-diagonal preconditioner
    identity_protein = sparse.identity(dirichlet_protein, dirichlet_protein, dirichlet_protein)
    identity_protein_diag = identity_protein.weak_form().to_sparse().diagonal()
    V_L_I_II_a_a_diag = laplace.single_layer(neumann_protein, dirichlet_protein, dirichlet_protein,
                                       assembler="only_diagonal_part").weak_form().get_diagonal()
    K_L_I_II_a_a_diag = laplace.double_layer(dirichlet_protein, dirichlet_protein, dirichlet_protein,
                                       assembler="only_diagonal_part").weak_form().get_diagonal()
    identity_stern = sparse.identity(dirichlet_stern, dirichlet_stern, dirichlet_stern)
    identity_stern_diag = identity_stern.weak_form().to_sparse().diagonal()
    V_L_II_b_b_diag = laplace.single_layer(neumann_stern, dirichlet_stern, dirichlet_stern,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    K_L_II_b_b_diag = laplace.double_layer(neumann_stern, dirichlet_stern, dirichlet_stern, 
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    V_Y_I_b_b_diag = modified_helmholtz.single_layer(neumann_stern, dirichlet_stern, dirichlet_stern, om,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()
    K_Y_I_b_b_diag = modified_helmholtz.double_layer(neumann_stern, dirichlet_stern, dirichlet_stern, om,
                                                   assembler="only_diagonal_part").weak_form().get_diagonal()

    diag11 = .5 * identity_protein_diag + K_L_I_II_a_a_diag
    diag12 = -V_L_I_II_a_a_diag
    diag21 = .5 * identity_protein_diag - K_L_I_II_a_a_diag
    diag22 = V_L_I_II_a_a_diag * f_op.weak_form().to_sparse().diagonal()

    d_aux = 1 / (diag22 - diag21 * diag12 / diag11)
    diag11_inv = 1 / diag11 + 1 / diag11 * diag12 * d_aux * diag21 / diag11
    diag12_inv = -1 / diag11 * diag12 * d_aux
    diag21_inv = -d_aux * diag21 / diag11
    diag22_inv = d_aux
    
    diag33 = .5 * identity_stern_diag +  K_L_II_b_b_diag
    diag34 = -V_L_II_b_b_diag 
    diag43 = .5 * identity_stern_diag - K_Y_I_b_b_diag
    diag44 = (1 / d2) * V_Y_I_b_b_diag * d1_op.weak_form().to_sparse().diagonal()
    
    d_aux = 1 / (diag44 - diag43 * diag34 / diag33)
    diag33_inv = 1 / diag33 + 1 / diag33 * diag34 * d_aux * diag43 / diag33
    diag34_inv = -1 / diag33 * diag34 * d_aux
    diag43_inv = -d_aux * diag43 / diag33
    diag44_inv = d_aux

    A = bmat([[diags(diag11_inv), diags(diag12_inv)],[diags(diag21_inv), diags(diag22_inv)]])
    B = bmat([[diags(diag33_inv), diags(diag34_inv)],[diags(diag43_inv), diags(diag44_inv)]])
              
    block_mat_precond = block_diag((A,B),format = 'csr')#.tocsr()
    return aslinearoperator(block_mat_precond)

In [8]:
for pqrindex in ['1','2']:
    for amino in range(len(res)):
        
        respath = os.path.join(CURRENTPATH,res[amino])
        protein_face_path = os.path.join(respath,res[amino]+'dielectric'+pqrindex+'.face')
        protein_vert_path = os.path.join(respath,res[amino]+'dielectric'+pqrindex+'.vert')
        grid_protein = import_msms_mesh(protein_face_path,protein_vert_path)
        grid_protein_dens = grid_protein.number_of_elements/sum(grid_protein.volumes)
        print(grid_protein_dens)

        stern_face_path = os.path.join(respath,res[amino]+'stern'+pqrindex+'.face')
        stern_vert_path = os.path.join(respath,res[amino]+'stern'+pqrindex+'.vert')
        grid_stern = import_msms_mesh(stern_face_path,stern_vert_path)
        grid_stern_dens = grid_stern.number_of_elements/sum(grid_stern.volumes)

        f_results.write(res[amino]+'Protein_Mesh_Number_of_Elements: '+str(grid_protein.number_of_elements)+'\n')
        f_results.write(res[amino]+'Protein_Mesh_Volume: '+str(sum(grid_protein.volumes))+'\n')
        f_results.write(res[amino]+'Protein_Mesh_Density: '+str(grid_protein.number_of_elements/sum(grid_protein.volumes))+'\n')
        f_results.write(res[amino]+'Stern_Mesh_Number_of_Elements: '+str(grid_stern.number_of_elements)+'\n')
        f_results.write(res[amino]+'Stern_Mesh_Volume: '+str(sum(grid_stern.volumes))+'\n')
        f_results.write(res[amino]+'Stern_Mesh_Density: '+str(grid_stern.number_of_elements/sum(grid_stern.volumes))+'\n')

        plot_mesh_to_file(os.path.join(respath,'results','protein_'),grid_protein)
        plot_mesh_to_file(os.path.join(respath,'results','stern_'),grid_stern)

        #Protein
        dirichlet_protein = bempp.api.function_space(grid_protein, "DP", 0)
        neumann_protein = bempp.api.function_space(grid_protein, "DP", 0)

        #Stern
        dirichlet_stern = bempp.api.function_space(grid_stern, "DP", 0)
        neumann_stern = bempp.api.function_space(grid_stern, "DP", 0)

        om = 0.125 #Debye lenght
        ep_i = 4
        ep_ii = 80
        ep_iii = 80

        precond_PB = block_diagonal_PB()
        
        #Operators:
        identity_protein = bempp.api.operators.boundary.sparse.identity(
            dirichlet_protein, dirichlet_protein, neumann_protein)

        K_L_I_a_a = bempp.api.operators.boundary.laplace.double_layer(
            dirichlet_protein, dirichlet_protein, neumann_protein)

        K_L_I_a_a_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
            dirichlet_protein, dirichlet_protein, neumann_protein)

        V_L_I_a_a = bempp.api.operators.boundary.laplace.single_layer(
            neumann_protein, dirichlet_protein, neumann_protein)

        K_L_II_a_a = bempp.api.operators.boundary.laplace.double_layer(
            dirichlet_protein, dirichlet_protein, neumann_protein)

        V_L_II_a_a = bempp.api.operators.boundary.laplace.single_layer(
            neumann_protein, dirichlet_protein, neumann_protein)

        K_L_II_a_b = bempp.api.operators.boundary.laplace.double_layer(
            dirichlet_stern, dirichlet_protein, neumann_protein) #OJO ACA

        V_L_II_a_b = bempp.api.operators.boundary.laplace.single_layer(
            neumann_stern, dirichlet_protein, neumann_protein) #OJO ACA

        K_L_II_b_a = bempp.api.operators.boundary.laplace.double_layer(
            dirichlet_protein, dirichlet_stern, neumann_stern) #OJO ACA

        V_L_II_b_a = bempp.api.operators.boundary.laplace.single_layer(
            neumann_protein, dirichlet_stern, neumann_stern) #OJO ACA

        identity_stern = bempp.api.operators.boundary.sparse.identity(
            dirichlet_stern, dirichlet_stern, neumann_stern)

        K_L_II_b_b = bempp.api.operators.boundary.laplace.double_layer(
            dirichlet_stern, dirichlet_stern, neumann_stern)

        V_L_II_b_b = bempp.api.operators.boundary.laplace.single_layer(
            neumann_stern, dirichlet_stern, neumann_stern)

        K_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.double_layer(
            dirichlet_stern, dirichlet_stern, neumann_stern, om)

        V_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.single_layer(
            neumann_stern, dirichlet_stern, neumann_stern, om)


        atoms=[] 
        pqrpath = os.path.join(respath,'j'+oneletter[amino]+pqrindex+'.pqr')
        pqr=open(pqrpath,'r')
        for line in pqr:
            if line[:4]=="ATOM": 
                line_ = line.split()
                x_c = [line_[6],line_[7],line_[8],line_[9]]
                x_c=[float(i) for i in x_c] 
                atoms.append(x_c)
        pqr.close()
        atoms=np.array(atoms) 

        # Se mapea la función analítica del lado derecho de la equación a la malla creada en Bempp:
        @bempp.api.real_callable
        def rhs_function(x, n, domain_index, result):

            result[:] = np.sum(atoms[:,3]/(4 * np.pi * ep_i * impl1(x-atoms[:,:3])))

        # Se crea una GridFunction conteniendo el lado derecho de la equación:
        rhs_1 = bempp.api.GridFunction(dirichlet_protein, fun=rhs_function)
        rhs_2 = np.zeros(neumann_protein.global_dof_count)
        rhs_3 = np.zeros(dirichlet_stern.global_dof_count)
        rhs_4 = np.zeros(neumann_stern.global_dof_count)
        rhs = np.concatenate([rhs_1.coefficients,rhs_2,rhs_3,rhs_4])

        blocked = bempp.api.BlockedOperator(4, 4)

        blocked[0,0] = 0.5 * identity_protein + K_L_I_a_a 
        blocked[0,1] = - V_L_I_a_a

        blocked[1,0] = 0.5 * identity_protein - K_L_II_a_a
        blocked[1,1] = (ep_i/ep_ii) * V_L_II_a_a
        blocked[1,2] = K_L_II_a_b 
        blocked[1,3] = - V_L_II_a_b

        blocked[2,0] = - K_L_II_b_a
        blocked[2,1] = (ep_i/ep_ii) * V_L_II_b_a 
        blocked[2,2] = 0.5 * identity_stern + K_L_II_b_b
        blocked[2,3] = - V_L_II_b_b

        blocked[3,2] = 0.5 * identity_stern - K_Y_III_b_b
        blocked[3,3] =  (ep_ii/ep_iii) * V_Y_III_b_b  

        blocked = blocked.strong_form()
        x = np.zeros(len(rhs))
        x, info_1 = gmres(blocked, rhs, x0 = x, M = precond_PB, maxiter=MAXITER_GM, tol = TOL)
        #x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

        solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                        x[(dirichlet_protein.global_dof_count+
                                                         neumann_protein.global_dof_count+
                                                          dirichlet_stern.global_dof_count):])

        solution_neumann_protein = bempp.api.GridFunction(neumann_protein, coefficients = 
                                                        x[dirichlet_protein.global_dof_count:
                                                         (dirichlet_protein.global_dof_count+
                                                          neumann_protein.global_dof_count)])

        solution_dirichlet_protein = bempp.api.GridFunction(dirichlet_protein, coefficients = 
                                                        x[:dirichlet_protein.global_dof_count])

        solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                            x[(dirichlet_protein.global_dof_count+
                                                             neumann_protein.global_dof_count):
                                                              (dirichlet_protein.global_dof_count+
                                                             neumann_protein.global_dof_count+
                                                              dirichlet_stern.global_dof_count)])

        S0_q = bempp.api.operators.potential.laplace.single_layer(neumann_protein, atoms[:,:3].transpose())
        D0_q = bempp.api.operators.potential.laplace.double_layer(dirichlet_protein, atoms[:,:3].transpose())
        phi_q = S0_q * solution_neumann_protein - D0_q * solution_dirichlet_protein
        en_sol = 2 * np.pi * 332.064 * np.sum(atoms[:,3] * phi_q).real

        #Se guardan soluciones en lista

        plot_solution_to_file(os.path.join(respath,'results','dirichlet_protein_PB_'+pqrindex),solution_dirichlet_protein)
        plot_solution_to_file(os.path.join(respath,'results','neumann_protein_PB_'+pqrindex),solution_neumann_protein)
        plot_solution_to_file(os.path.join(respath,'results','dirichlet_stern_PB_'+pqrindex),solution_dirichlet_stern)
        plot_solution_to_file(os.path.join(respath,'results','neumann_stern_PB_'+pqrindex),solution_neumann_stern)
        f_results.write(res[amino]+'_PB_Solvation_Energy_'+pqrindex+': '+str(en_sol))
        f_results.write(res[amino]+'_PB_Phi_Dielectric'+pqrindex+': '+str(np.mean(solution_dirichlet_protein.coefficients)))
        f_results.write(res[amino]+'_PB_dPhi_Dielectric'+pqrindex+': '+str(np.mean(solution_neumann_protein.coefficients)))
        f_results.write(res[amino]+'_PB_Phi_Stern'+pqrindex+': '+str(np.mean(solution_dirichlet_stern.coefficients)))
        f_results.write(res[amino]+'_PB_dPhi_Stern'+pqrindex+': '+str(np.mean(solution_neumann_stern.coefficients)))
        
        np.savetxt(os.path.join(respath,'results','dirichlet_protein_PB_'+pqrindex+'.csv'), solution_dirichlet_protein.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','neumann_protein_PB_'+pqrindex+'.csv'), solution_neumann_protein.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','dirichlet_stern_PB_'+pqrindex+'.csv'), solution_dirichlet_stern.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','neumann_stern_PB_'+pqrindex+'.csv'), solution_neumann_stern.coefficients, delimiter=",")
        

        @bempp.api.real_callable
        def en_function(x, n, domain_index, result):

            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms[:,:3])/
                                         (4 * np.pi * impl1(x-atoms[:,:3])**3))*
                                      (atoms[:,3])))

        en_1 = bempp.api.GridFunction(dirichlet_protein, fun=en_function)

        def f(sigma,fun):
            Ksigma = K_L_I_a_a_adjoint * sigma
            En = fun - Ksigma
            mu = - alpha * np.tanh(- gamma)
            h = alpha * np.tanh(beta * En.coefficients - gamma) + mu
            f = ep_i/(ep_ii - ep_i) - h
            f_div = f/(1+f)
            return f_div 
        d2 = 1
        qtotal = np.sum(atoms[:,3])
        d1 = - qtotal/ep_ii 
        sigma = bempp.api.GridFunction(dirichlet_protein, coefficients = np.zeros(
            dirichlet_protein.global_dof_count))

        #SLIC
        for i in range(ITER): 
            blocked = bempp.api.BlockedOperator(4, 4)
            if i != 0:
                d2 = solution_neumann_stern.integrate()[0] 
                f_it = f(sigma,en_1)
                f_fun = bempp.api.GridFunction(neumann_protein, coefficients = f_it)
                f_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun, 
                                        neumann_protein, neumann_protein, neumann_protein)
                blocked[1,1] =  V_L_II_a_a * f_op
                blocked[2,1] =  V_L_II_b_a * f_op
                precond_SLIC = block_diagonal_SLIC()

            else:
                blocked[1,1] =  V_L_II_a_a
                blocked[2,1] =  V_L_II_b_a
                precond_SLIC = precond_PB


            blocked[0,0] = 0.5 * identity_protein + K_L_I_a_a 
            blocked[0,1] = - V_L_I_a_a

            blocked[1,0] = 0.5 * identity_protein - K_L_II_a_a
            blocked[1,2] = K_L_II_a_b 
            blocked[1,3] = - V_L_II_a_b

            blocked[2,0] = - K_L_II_b_a
            blocked[2,2] = 0.5 * identity_stern + K_L_II_b_b
            blocked[2,3] = - V_L_II_b_b

            blocked[3,2] = 0.5 * identity_stern - K_Y_III_b_b
            blocked[3,3] =  V_Y_III_b_b * (d1/d2) 

            blocked = blocked.strong_form()

            x, info_1 = gmres(blocked, rhs, x0 = x, M = precond_SLIC, maxiter=MAXITER_GM, tol = TOL)
            #x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

            solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                            x[(dirichlet_protein.global_dof_count+
                                                             neumann_protein.global_dof_count+
                                                              dirichlet_stern.global_dof_count):])
            #Sigma Ec. 46
            solution_neumann_protein = bempp.api.GridFunction(neumann_protein, coefficients = 
                                                            x[dirichlet_protein.global_dof_count:
                                                             (dirichlet_protein.global_dof_count+
                                                              neumann_protein.global_dof_count)])

            solution_dirichlet_protein = bempp.api.GridFunction(dirichlet_protein, coefficients = 
                                                            x[:dirichlet_protein.global_dof_count])

            solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                x[(dirichlet_protein.global_dof_count+
                                                 neumann_protein.global_dof_count):
                                                  (dirichlet_protein.global_dof_count+
                                                 neumann_protein.global_dof_count+
                                                  dirichlet_stern.global_dof_count)])

            rhs_sigma = V_L_I_a_a * solution_neumann_protein - (- 0.5 * identity_protein + K_L_I_a_a) * solution_dirichlet_protein

            sigma, info_2 = bempp.api.linalg.gmres(V_L_I_a_a, rhs_sigma, 
                                                   tol=TOL, maxiter = MAXITER_GM) 

            S0_q = bempp.api.operators.potential.laplace.single_layer(neumann_protein, atoms[:,:3].transpose())
            D0_q = bempp.api.operators.potential.laplace.double_layer(dirichlet_protein, atoms[:,:3].transpose())
            phi_q = S0_q * solution_neumann_protein - D0_q * solution_dirichlet_protein
            en_sol = 2 * np.pi * 332.064 * np.sum(atoms[:,3] * phi_q).real
            print(res[amino],pqrindex,'SLIC',i,en_sol)


        plot_solution_to_file(os.path.join(respath,'results','dirichlet_protein_SLIC_'+pqrindex),solution_dirichlet_protein)
        plot_solution_to_file(os.path.join(respath,'results','neumann_protein_SLIC_'+pqrindex),solution_neumann_protein)
        plot_solution_to_file(os.path.join(respath,'results','dirichlet_stern_SLIC_'+pqrindex),solution_dirichlet_stern)
        plot_solution_to_file(os.path.join(respath,'results','neumann_stern_SLIC_'+pqrindex),solution_neumann_stern)
        f_results.write(res[amino]+'_SLIC_Solvation_Energy_'+pqrindex+': '+str(en_sol))
        f_results.write(res[amino]+'_SLIC_Phi_Dielectric'+pqrindex+': '+str(np.mean(solution_dirichlet_protein.coefficients)))
        f_results.write(res[amino]+'_SLIC_dPhi_Dielectric'+pqrindex+': '+str(np.mean(solution_neumann_protein.coefficients)))
        f_results.write(res[amino]+'_SLIC_Phi_Stern'+pqrindex+': '+str(np.mean(solution_dirichlet_stern.coefficients)))
        f_results.write(res[amino]+'_SLIC_dPhi_Stern'+pqrindex+': '+str(np.mean(solution_neumann_stern.coefficients)))
        
        np.savetxt(os.path.join(respath,'results','dirichlet_protein_SLIC_'+pqrindex+'.csv'), solution_dirichlet_protein.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','neumann_protein_SLIC_'+pqrindex+'.csv'), solution_neumann_protein.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','dirichlet_stern_SLIC_'+pqrindex+'.csv'), solution_dirichlet_stern.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','neumann_stern_SLIC_'+pqrindex+'.csv'), solution_neumann_stern.coefficients, delimiter=",")
        
        #SLIC PROPOSED
        d2 = 1

        @bempp.api.real_callable
        def d1_function(x, n, domain_index, result):
            result[:] = np.sum(atoms[:,3]/(impl1(x-atoms[:,:3])))

        d1_fun = bempp.api.GridFunction(neumann_stern, fun = d1_function) 
        if qtotal == 0:
            print('Carga Cero!')
            d1_mat = -(1/ep_ii) * d1_fun.coefficients
        else:
            d1_mat = -(qtotal/ep_ii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
        d1_gridfun = bempp.api.GridFunction(neumann_stern, coefficients = d1_mat)
        d1_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(d1_gridfun, 
                                                    neumann_stern, neumann_stern, neumann_stern)            

        sigma = bempp.api.GridFunction(dirichlet_protein, coefficients = np.zeros(dirichlet_protein.global_dof_count))

        for i in range(ITER): 
            blocked = bempp.api.BlockedOperator(4, 4)
            if i != 0:
                d2 = solution_neumann_stern.integrate()[0] 
                f_it = f(sigma,en_1)
                f_fun = bempp.api.GridFunction(neumann_protein, coefficients = f_it)
                f_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun, 
                                        neumann_protein, neumann_protein, neumann_protein)
                blocked[1,1] =  V_L_II_a_a * f_op
                blocked[2,1] =  V_L_II_b_a * f_op
                blocked[3,3] =  (1/d2) * V_Y_III_b_b * d1_op
                precond_SLIC_PROP = block_diagonal_SLIC_PROP()
            else:
                blocked[1,1] =  V_L_II_a_a
                blocked[2,1] =  V_L_II_b_a
                blocked[3,3] =  (1/d2) * V_Y_III_b_b 
                precond_SLIC_PROP = precond_PB


            blocked[0,0] = 0.5 * identity_protein + K_L_I_a_a 
            blocked[0,1] = - V_L_I_a_a

            blocked[1,0] = 0.5 * identity_protein - K_L_II_a_a
            blocked[1,2] = K_L_II_a_b 
            blocked[1,3] = - V_L_II_a_b

            blocked[2,0] = - K_L_II_b_a
            blocked[2,2] = 0.5 * identity_stern + K_L_II_b_b
            blocked[2,3] = - V_L_II_b_b

            blocked[3,2] = 0.5 * identity_stern - K_Y_III_b_b

            blocked = blocked.strong_form()

            x, info_1 = gmres(blocked, rhs, x0 = x, M = precond_SLIC_PROP, maxiter=MAXITER_GM, tol = TOL)
            #x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

            solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                            x[(dirichlet_protein.global_dof_count+
                                                             neumann_protein.global_dof_count+
                                                              dirichlet_stern.global_dof_count):])
            #Sigma Ec. 46
            solution_neumann_protein = bempp.api.GridFunction(neumann_protein, coefficients = 
                                                            x[dirichlet_protein.global_dof_count:
                                                             (dirichlet_protein.global_dof_count+
                                                              neumann_protein.global_dof_count)])

            solution_dirichlet_protein = bempp.api.GridFunction(dirichlet_protein, coefficients = 
                                                            x[:dirichlet_protein.global_dof_count])

            solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                x[(dirichlet_protein.global_dof_count+
                                                 neumann_protein.global_dof_count):
                                                  (dirichlet_protein.global_dof_count+
                                                 neumann_protein.global_dof_count+
                                                  dirichlet_stern.global_dof_count)])

            rhs_sigma = V_L_I_a_a * solution_neumann_protein - (- 0.5 * identity_protein + K_L_I_a_a) * solution_dirichlet_protein

            sigma, info_2 = bempp.api.linalg.gmres(V_L_I_a_a, rhs_sigma, 
                                                   tol=TOL, maxiter = MAXITER_GM) 

            S0_q = bempp.api.operators.potential.laplace.single_layer(neumann_protein, atoms[:,:3].transpose())
            D0_q = bempp.api.operators.potential.laplace.double_layer(dirichlet_protein, atoms[:,:3].transpose())
            phi_q = S0_q * solution_neumann_protein - D0_q * solution_dirichlet_protein
            en_sol = 2 * np.pi * 332.064 * np.sum(atoms[:,3] * phi_q).real
            print(res[amino],pqrindex,'SLICPROP',i,en_sol)

        plot_solution_to_file(os.path.join(respath,'results','dirichlet_protein_SLICPROP_'+pqrindex),solution_dirichlet_protein)
        plot_solution_to_file(os.path.join(respath,'results','neumann_protein_SLICPROP_'+pqrindex),solution_neumann_protein)
        plot_solution_to_file(os.path.join(respath,'results','dirichlet_stern_SLICPROP_'+pqrindex),solution_dirichlet_stern)
        plot_solution_to_file(os.path.join(respath,'results','neumann_stern_SLICPROP_'+pqrindex),solution_neumann_stern)
        f_results.write(res[amino]+'_SLICPROP_Solvation_Energy_'+pqrindex+': '+str(en_sol))
        f_results.write(res[amino]+'_SLICPROP_Phi_Dielectric'+pqrindex+': '+str(np.mean(solution_dirichlet_protein.coefficients)))
        f_results.write(res[amino]+'_SLICPROP_dPhi_Dielectric'+pqrindex+': '+str(np.mean(solution_neumann_protein.coefficients)))
        f_results.write(res[amino]+'_SLICPROP_Phi_Stern'+pqrindex+': '+str(np.mean(solution_dirichlet_stern.coefficients)))
        f_results.write(res[amino]+'_SLICPROP_dPhi_Stern'+pqrindex+': '+str(np.mean(solution_neumann_stern.coefficients)))
        
        np.savetxt(os.path.join(respath,'results','dirichlet_protein_SLICPROP_'+pqrindex+'.csv'), solution_dirichlet_protein.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','neumann_protein_SLICPROP_'+pqrindex+'.csv'), solution_neumann_protein.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','dirichlet_stern_SLICPROP_'+pqrindex+'.csv'), solution_dirichlet_stern.coefficients, delimiter=",")
        np.savetxt(os.path.join(respath,'results','neumann_stern_SLICPROP_'+pqrindex+'.csv'), solution_neumann_stern.coefficients, delimiter=",")
        

22.02890036163867


/home/kennethstyles/miniconda3/envs/bempp_prod_openmm/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



arg 1 SLIC 0 -9.511924590745394


/home/kennethstyles/miniconda3/envs/bempp_prod_openmm/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



arg 1 SLIC 1 -19.609678325194473
arg 1 SLIC 2 -19.457902309567576
arg 1 SLIC 3 -19.45250042236227
arg 1 SLIC 4 -19.45253551102548
arg 1 SLIC 5 -19.452537543575577
arg 1 SLIC 6 -19.452537543575577
arg 1 SLIC 7 -19.452537543575577
arg 1 SLIC 8 -19.452537543575577
arg 1 SLIC 9 -19.452537543575577
arg 1 SLIC 10 -19.452537543575577
arg 1 SLIC 11 -19.452537543575577
arg 1 SLIC 12 -19.452537543575577
arg 1 SLIC 13 -19.452537543575577
arg 1 SLIC 14 -19.452537543575577
arg 1 SLIC 15 -19.452537543575577
arg 1 SLIC 16 -19.452537543575577
arg 1 SLIC 17 -19.452537543575577
arg 1 SLIC 18 -19.452537543575577
arg 1 SLIC 19 -19.452537543575577


...

tyr 2 SLIC 0 -9.763649632459602


/home/kennethstyles/miniconda3/envs/bempp_prod_openmm/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



tyr 2 SLIC 1 -29.272673771289952
tyr 2 SLIC 2 -28.735070558760878
tyr 2 SLIC 3 -28.727680714520744
tyr 2 SLIC 4 -28.727968675175113
tyr 2 SLIC 5 -28.727967376974725
tyr 2 SLIC 6 -28.727967376974725
tyr 2 SLIC 7 -28.727967376974725
tyr 2 SLIC 8 -28.727967376974725
tyr 2 SLIC 9 -28.727967376974725
tyr 2 SLIC 10 -28.727967376974725
tyr 2 SLIC 11 -28.727967376974725
tyr 2 SLIC 12 -28.727967376974725
tyr 2 SLIC 13 -28.727967376974725
tyr 2 SLIC 14 -28.727967376974725
tyr 2 SLIC 15 -28.727967376974725
tyr 2 SLIC 16 -28.727967376974725
tyr 2 SLIC 17 -28.727967376974725
tyr 2 SLIC 18 -28.727967376974725
tyr 2 SLIC 19 -28.727967376974725


/home/kennethstyles/miniconda3/envs/bempp_prod_openmm/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



tyr 2 SLICPROP 0 -3.2200869307115045
tyr 2 SLICPROP 1 -29.37614999458571
tyr 2 SLICPROP 2 -28.556447835707246
tyr 2 SLICPROP 3 -28.53880087475876
tyr 2 SLICPROP 4 -28.539464198713816
tyr 2 SLICPROP 5 -28.53946746541224
tyr 2 SLICPROP 6 -28.53946696969817
tyr 2 SLICPROP 7 -28.53946696969817
tyr 2 SLICPROP 8 -28.53946696969817
tyr 2 SLICPROP 9 -28.53946696969817
tyr 2 SLICPROP 10 -28.53946696969817
tyr 2 SLICPROP 11 -28.53946696969817
tyr 2 SLICPROP 12 -28.53946696969817
tyr 2 SLICPROP 13 -28.53946696969817
tyr 2 SLICPROP 14 -28.53946696969817
tyr 2 SLICPROP 15 -28.53946696969817
tyr 2 SLICPROP 16 -28.53946696969817
tyr 2 SLICPROP 17 -28.53946696969817
tyr 2 SLICPROP 18 -28.53946696969817
tyr 2 SLICPROP 19 -28.53946696969817


In [9]:
f_results.close()